## 🛍️ AI Assistant for E-Commerce Customer Support

You've just joined **Trendora**, a fast-growing e-commerce startup known for its trendy gadgets and same-day delivery promise. The support team is overwhelmed with customer messages—ranging from product inquiries to returns and even pirate-style rants from angry customers.

Your manager wants to **automate parts of the customer service pipeline** using local language models to save on API costs and increase data privacy.

You’ll be working with **LangChain + Ollama**, using **Meta's LLaMA 3.2 model** running locally.

Your mission is to:

- Generate consistent and polite responses to customer emails using custom prompt templates.
- Translate angry or informal emails into professional English.
- Extract structured info (like refund intent, product issue, delivery time) from unstructured reviews.
- Make sure everything runs locally for cost and compliance reasons.

Let’s dive into it and start building a smart, private-first language assistant for Trendora’s e-commerce operations.

Let's install the needed modules

In [ ]:
!pip install langchain langchain_core langchain_ollama

Let's import the needed modules

In [ ]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import PromptTemplate

llm = OllamaLLM(model="llama3.2:latest")
llm

Let’s start by sending a basic prompt to our locally running model to see if it works correctly.

In [ ]:
prompt = "What is 1 + 1?"

response = llm.invoke(prompt)
print(response)

Your boss just asked you to handle a customer complaint. A customer sent a very upset message about a defective gadget. Your task is to translate their message into a calm and polite tone.

In [ ]:
customer_email = """
I am furious! I ordered your wireless speaker and the battery died in two hours. This is ridiculous. I want a refund NOW!
"""

style = "American English in a calm and respectful tone"

prompt = f"""Translate the text delimited by triple backticks into a style that is {style}.
text: ```{customer_email}```"""

response = llm.invoke(prompt)
print(response)

We can see that the model successfully rewrote the email in a more professional and respectful tone while keeping the original message intact.

Now, what if your boss asks you to process multiple angry emails every day?

Instead of writing a new prompt each time, you can use LangChain’s `PromptTemplate` to build a reusable template. let me show you how it works.

In [ ]:
template_string = """Translate the text delimited by triple backticks into a style that is {style}. text: ```{text}```"""

prompt_template = PromptTemplate.from_template(template_string)

formatted_prompt = prompt_template.format(
	style=style,
	text=customer_email
)

response = llm.invoke(formatted_prompt)
print(response)

This way, we can use the `format` function to fill in the prompt template whenever needed.

This time, imagine you’ve received an email from a customer who’s having trouble with a recently purchased smartwatch. Instead of rewriting the prompt from scratch, you simply plug the message and desired tone into the template. here is how:

In [ ]:
service_reply = """Dear customer, thank you for contacting us. We're sorry to hear that your smartwatch is not charging properly. 
Please try using a different charging cable and make sure the contact points are clean. 
If the issue persists, you can request a replacement through our support center."""

service_style_friendly = "a warm and understanding tone in American English, suitable for customer support"

formatted_service_prompt = prompt_template.format(
	style=service_style_friendly,
	text=service_reply
)

response = llm.invoke(formatted_service_prompt)
print(response)

Now You’re tasked with extracting structured information from product reviews — such as whether the item was a gift, how long delivery took, and how the customer perceived the product’s price or value.

Let me show you how you can get your response with a json structure.

In [ ]:
customer_review = """
This air fryer is amazing. My husband got it as a gift for our anniversary and it arrived within 3 days. It’s a bit pricey, but honestly worth every penny.
"""

review_template = """For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

Format the output as JSON.

text: {text}
"""

review_prompt = PromptTemplate.from_template(review_template)
formatted_review = review_prompt.format(text=customer_review)

response = llm.invoke(formatted_review)
print(response)

Your boss wants to know: what if the LLM returns a JSON response with the wrong structure for example missing keys or inconsistent formatting?

To make the output reliable and consistent, we need a way to enforce the correct JSON structure every time.

To do this we can use `Langchain Output Parsers`. let's import them.

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

To make sure the JSON output from the language model always follows a consistent structure, we can define a set of response schemas using `ResponseSchema`. Each schema specifies a key we expect in the output and describes what it should contain.

This allows us to generate a clear set of formatting instructions that guide the model to return data in a predictable and structured way — making it much easier to parse the results programmatically.

Let’s define the expected fields:  
- `gift`: whether the item was purchased as a gift  
- `delivery_days`: how many days it took to arrive  
- `price_value`: comments related to price or value

So now, let's do it...

In [ ]:
gift_schema = ResponseSchema(
	name="gift",
	description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown."
)

delivery_days_schema = ResponseSchema(
	name="delivery_days",
	description="How many days did it take for the product to arrive? If this info is not found, output -1."
)

price_value_schema = ResponseSchema(
	name="price_value",
	description="Extract any sentences about the value or price, output as comma separated list."
)

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

Now let's have a look at the `response` with json object again.

In [ ]:
print(response)

Now we can use the output parser to parse the response into a `python dictionary`.

In [ ]:
output_dict = output_parser.parse(response)
print("parsed output:", output_dict)
print("type of the output: " , type(output_dict))

So as you can see result is a structured and easy-to-use Python dictionary (`dict`), which can now be passed into databases, used for analytics, or integrated into automated workflows.